In [36]:
import pandas as pd
import numpy as np
import folium

In [37]:
df = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            132 non-null    int64  
 1   공원명           132 non-null    object 
 2   공원개요          131 non-null    object 
 3   면적            129 non-null    object 
 4   개원일           122 non-null    object 
 5   주요시설          124 non-null    object 
 6   주요식물          88 non-null     object 
 7   안내도           102 non-null    object 
 8   오시는길          115 non-null    object 
 9   이용시참고사항       120 non-null    object 
 10  이미지           132 non-null    object 
 11  지역            131 non-null    object 
 12  공원주소          132 non-null    object 
 13  관리부서          132 non-null    object 
 14  전화번호          132 non-null    object 
 15  X좌표(GRS80TM)  122 non-null    float64
 16  Y좌표(GRS80TM)  122 non-null    float64
 17  X좌표(WGS84)    131 non-null    float64
 18  Y좌표(WGS84)    131 non-null    

In [38]:
df = df[['공원명','면적','공원주소','X좌표(WGS84)','Y좌표(WGS84)']] # X=위도, Y=경도
df.columns = ['공원명','면적','주소','경도','위도'] # 이름 쉽게 바꾸기
df.head(3)

,공원명,면적,주소,경도,위도
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690m²,경기도 과천시 대공원광장로 102,127.019846,37.426449


In [39]:
df.dropna(subset=['면적','경도'], inplace=True) # 면적, 경도가 없는 데이터는 과감하게 삭제함(찾을 수 있으면 찾기)
df.isna().sum().sum()

0

- 면적에 따라 공원의 크기를 지도에 표시

In [40]:
# 데이터를 요리조리 확인해본다(구조, 내용 확인)

df.면적.head(10)   # 휴양및 편의시설 있네-> 삭제할 것
df.면적.head(30).tail(10)   # '총 ~m2' 데이터 보임

20      113021.7m2
21        201779㎥ 
22      258991㎡   
23    10420819.08㎡
24       171294.1㎥
25        총 15000㎡
26         297926㎡
27         411972㎡
28          98470㎡
29      2038741.3㎡
Name: 면적, dtype: object

In [41]:
df.면적[6]  # '휴양...' 삭제
df.drop([6], inplace=True)

In [42]:
# 시작할 때 있는 '총 ' 지우기 (총 144m)
df.면적 = df.면적.str.replace('총 ', '')

```구분자 ㎡, m, ㎥ 앞의 숫자만 추출하기```

In [43]:
# split() 설명 -구분자가 여러개인 경우
'a,b.c'.split(',.') # ['a,b.c']
'a,b.c'.split('[,.]') # 안먹음

import re
re.split('[,.]', 'a,b.c')

['a', 'b', 'c']

In [44]:
# 구분자 ㎡, m, ㎥ 앞의 숫자만 추출하기
df.면적 = df.면적.apply(lambda x: re.split('[㎡m㎥]', x)[0])   # split결과의 첫번째[0] 값
df.면적.head()

0    2896887
1      80683
2    9132690
3     480994
4    2284085
Name: 면적, dtype: object

In [45]:
#df.면적 = df.면적.astype(int)   # 안됨 - 데이터 중 8948.1같은 실수가 있음
df.면적 = df.면적.astype(float).astype(int)
df.head()

,공원명,면적,주소,경도,위도
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805


- 공원 크기에 따라 분류

In [66]:
# 분류 기준은 qcut쓰든 마음대로
# 썜은 소/중/대로 나눔
# 소형 < 100000, 중형 < 1000000, 대형
criteria = [0, 100000, 1000000, 20000000]   # 대형 수치는 아직 모름-> 범위 크게 잡고 시작
labels = ['소형', '중형', '대형']   # 분류 기준
size_info = [3, 7, 15]              # CircleMarker의 크기 지정
df['분류'] = pd.cut(df.면적, criteria, labels=labels)
df['크기'] = pd.cut(df.면적, criteria, labels=size_info)
df.head()

,공원명,면적,주소,경도,위도,분류,size,크기
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,대형,896,15
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,소형,896,3
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449,대형,896,15
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,중형,896,7
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,대형,896,15


In [61]:
# df.크기 = df.크기.astype(int) -> 아래 for문에서 어차피 int를 줘야 함. pass

In [65]:
# type(df.크기[0])

numpy.int32

In [67]:
df.to_csv('data/서울공원요약.csv', index=False)

- 공원 시각화

In [68]:
park = folium.Map([37.55, 126.98], zoom_start=11)
for i in df.index:
    folium.CircleMarker(
        location=[df.위도[i], df.경도[i]],
        radius=int(df.크기[i]),   # 여기서 int를 붙여야 함(위에서 한번 정리했더라도)
        tooltip=df.공원명[i],
        popup=folium.Popup(df.주소[i], max_width=200),
        color='crimson', fill_color='crimson'
    ).add_to(park)
    
title_html = '<h3 align="center" style="font-size:20px">서울시 공원 현황</h3>'
park.get_root().html.add_child(folium.Element(title_html))
park